In [4]:
import os
import uuid
import multiprocessing
from functools import reduce

import cv2

import numpy as np

import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset, sampler

from matplotlib import pyplot as plt

from pdb import set_trace

from ddh import *
from dataset import *
from logger import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# reset the data

undo_create_set("val")
undo_create_set("test")
create_set("val")
create_set("test")

In [15]:
# switch to the appropriate device

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = torch.device("cpu")

print(device)

cuda


In [8]:
# hyperparameters

num_epochs = 1000

optimizer_params = {
    "lr": 1e-5,
    "weight_decay": 0.0
}
custom_params = {
    "beta": 0.0 # regularization to guide bits to either 1 or -1
}

# how many bits to map the images to with the network
hash_dim = 48

In [9]:
# add data transformation here

transform = T.Compose([
    T.ToPILImage(),
    T.Resize((32, 32)),
    T.ToTensor()
])

In [10]:
# build the dataset 

data_train = FaceScrubDataset(type="hash_label",
                              mode="train",
                              transform=transform,
                              hash_dim=hash_dim)
data_val = FaceScrubDataset(type="hash_label",
                            mode="val",
                            transform=transform,
                            hash_dim=hash_dim)
data_test = FaceScrubDataset(type="hash_label",
                             mode="test",
                             transform=transform,
                             hash_dim=hash_dim)

In [33]:
# setting up the data loader

batch_size = {
    "train": 1024,
    "val": 1024,
    "test": 1024
}

loader_params = {
    "shuffle": True,
    "num_workers": multiprocessing.cpu_count() - 2
}

loader_train = DataLoader(data_train,
                          batch_size=batch_size["train"],
                          **loader_params)
loader_val = DataLoader(data_val,
                          batch_size=batch_size["val"],
                          **loader_params)
loader_test = DataLoader(data_test,
                          batch_size=batch_size["test"],
                          **loader_params)

In [ ]:
# components

model = DiscriminativeDeepHashing(hash_dim=hash_dim)
model = model.to(device=device)
optimizer = optim.Adam(model.parameters(), **optimizer_params)

run_id = uuid.uuid4().hex.upper()[0:6]
checkpoint_path = os.getcwd() + "/models/{}_best_weights.pt".format(run_id)

# training code

train_losses = []
train_acc = []
val_acc = []

with Logger(write_to_file=False) as logger:
    logger.write("Starting run for {}".format(run_id))
    logger.write("====== START ======")

    for epoch in range(num_epochs):
        epoch_train_losses = []
        epoch_train_data_correct = []
        epoch_train_data_total = []
        epoch_train_bits_correct = []
        epoch_train_bits_total = []

        for num_iter, (X, y) in enumerate(loader_train):
            # set model to train mode
            model.train()
            # forward pass
            batch = X.to(device).float()
            label = y.to(device).float()
            outputs = model(batch)
            # calculating loss
            loss = F.binary_cross_entropy_with_logits(outputs, label)
            # loss to encourage code to be closer to -1 and 1
            loss += custom_params["beta"] * \
                        torch.abs((torch.abs(outputs)-1).sum()) / len(X)
            # back propagate
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # calculate accuracy over training data
            label = label.byte()
            output_hash = (outputs.data > 0).to(device)
            # a classification is considered correct if the output matches the target bits exactly
            num_correct = ((output_hash == label).sum(1) == hash_dim).sum().item()
            # the number of bits that are correct
            num_bits_correct = (output_hash == label).sum().item()
            num_bits_total = reduce(lambda x, y: x*y, output_hash.shape)
            # calculate the percentage
            train_data_acc = num_correct * 100.0 / len(X)
            train_bits_acc = num_bits_correct * 100.0 / num_bits_total
            # save statistics
            epoch_train_losses.append(loss.item())
            epoch_train_data_correct.append(num_correct)
            epoch_train_data_total.append(len(X))
            epoch_train_bits_correct.append(num_bits_correct)
            epoch_train_bits_total.append(num_bits_total)

            if num_iter % 10 == 0:
                print(
                    "TRAIN epoch {} ".format(epoch) +
                    "iter {}: ".format(num_iter) +
                    "loss - {:.10f}, ".format(loss.item()) +
                    "correct/total - {}/{} ({:.2f}%), "
                        .format(num_correct, len(X), train_data_acc) +
                    "bits/total - {}/{} ({:.2f}%). "
                        .format(num_bits_correct, num_bits_total, train_bits_acc)
                )
        
        epoch_val_data_correct = []
        epoch_val_bits_correct = []
        epoch_val_data_total = []
        epoch_val_bits_total = []
        
        for num_iter, (X, y) in enumerate(loader_val):
            model.eval()
            batch = X.float().to(device)
            label = y.float().to(device)
            outputs = model(batch)
            # calculate accuracy over validation data
            label = label.byte()
            output_hash = (outputs.data > 0).to(device)
            # number of data points correct
            num_correct = ((output_hash == label).sum(1) == hash_dim).sum().item()
            # number of bits correct
            num_bits_correct = (output_hash == label).sum().item()
            num_bits_total = reduce(lambda x, y: x*y, output_hash.shape)
            # append to history
            epoch_val_data_correct.append(num_correct)
            epoch_val_data_total.append(len(X))
            epoch_val_bits_correct.append(num_bits_correct)
            epoch_val_bits_total.append(num_bits_total)
            
            
            # calculate the loss and accuracy over the whole dataset by averaging them
            final_train_loss = reduce(lambda x,y: x+y, epoch_train_losses) / len(epoch_train_losses)
            final_train_data_correct = reduce(lambda x,y: x+y, epoch_train_data_correct) / len(epoch_train_data_correct)
            final_train_data_total = reduce(lambda x,y: x+y, epoch_train_data_total) / len(epoch_train_data_total)
            final_train_bits_correct = reduce(lambda x,y: x+y, epoch_train_bits_correct) / len(epoch_train_bits_correct)
            final_train_bits_total = reduce(lambda x,y: x+y, epoch_train_bits_total) / len(epoch_train_bits_total)
        
            final_val_data_correct = reduce(lambda x,y: x+y, epoch_val_data_correct) / len(epoch_val_data_correct)
            final_val_data_total = reduce(lambda x,y: x+y, epoch_val_data_total) / len(epoch_val_data_total)
            final_val_bits_correct = reduce(lambda x,y: x+y, epoch_val_bits_correct) / len(epoch_val_bits_correct)
            final_val_bits_total = reduce(lambda x,y: x+y, epoch_val_bits_total) / len(epoch_val_bits_total)
        
        print(
            "TRAIN epoch {} completed! Avg train loss - {:.6f}".format(epoch, final_train_loss) +
            "\n\tTRAIN data correct/total - {}/{} ({:.2f}%), bits correct/total - {}/{} ({:.2f}%)"
                .format(round(final_train_data_correct), round(final_train_data_total),
                        final_train_data_correct * 100.0 / final_train_data_total,
                        round(final_train_bits_correct), round(final_train_bits_total), 
                        final_train_bits_correct * 100.0 /  final_train_bits_total) +
            "\n\tVAL   data correct/total - {}/{} ({:.2f}%), bits correct/total - {}/{} ({:.2f}%)"
                .format(round(final_val_data_correct), round(final_val_data_total), 
                        final_val_data_correct * 100.0 / final_val_data_total,
                        round(final_val_bits_correct), round(final_val_bits_total),
                        final_val_bits_correct * 100.0 / final_val_bits_total)
        )
            
    logger.write("====== END ======")
    logger.write("Completed run for {}".format(run_id))

Starting run for C47D02
====== START ======
TRAIN epoch 0 iter 0: loss - 0.6941315532, correct/total - 0/1024 (0.00%), bits/total - 24954/49152 (50.77%). 
TRAIN epoch 0 iter 10: loss - 0.6931667328, correct/total - 0/1024 (0.00%), bits/total - 25317/49152 (51.51%). 
TRAIN epoch 0 iter 20: loss - 0.6929103732, correct/total - 0/1024 (0.00%), bits/total - 25268/49152 (51.41%). 
TRAIN epoch 0 iter 30: loss - 0.6923102140, correct/total - 0/1024 (0.00%), bits/total - 25071/49152 (51.01%). 
TRAIN epoch 0 iter 40: loss - 0.6913444996, correct/total - 0/1024 (0.00%), bits/total - 25246/49152 (51.36%). 
TRAIN epoch 0 iter 50: loss - 0.6906951666, correct/total - 0/1024 (0.00%), bits/total - 25242/49152 (51.35%). 
TRAIN epoch 0 completed! Avg train loss - 0.692343
	TRAIN data correct/total - 0/1021 (0.00%), bits correct/total - 25108/49027 (51.21%)
	VAL   data correct/total - 0/883 (0.00%), bits correct/total - 21741/42400 (51.28%)
TRAIN epoch 1 iter 0: loss - 0.6904133558, correct/total - 0/10

TRAIN epoch 9 iter 20: loss - 0.6792839766, correct/total - 0/1024 (0.00%), bits/total - 26158/49152 (53.22%). 
TRAIN epoch 9 iter 30: loss - 0.6795315146, correct/total - 0/1024 (0.00%), bits/total - 26030/49152 (52.96%). 
TRAIN epoch 9 iter 40: loss - 0.6793741584, correct/total - 0/1024 (0.00%), bits/total - 25987/49152 (52.87%). 
TRAIN epoch 9 iter 50: loss - 0.6793092489, correct/total - 0/1024 (0.00%), bits/total - 25991/49152 (52.88%). 
TRAIN epoch 9 completed! Avg train loss - 0.679477
	TRAIN data correct/total - 0/1021 (0.00%), bits correct/total - 25973/49027 (52.98%)
	VAL   data correct/total - 0/883 (0.00%), bits correct/total - 22423/42400 (52.88%)
TRAIN epoch 10 iter 0: loss - 0.6792353988, correct/total - 0/1024 (0.00%), bits/total - 26071/49152 (53.04%). 
TRAIN epoch 10 iter 10: loss - 0.6794235110, correct/total - 0/1024 (0.00%), bits/total - 25949/49152 (52.79%). 
TRAIN epoch 10 iter 20: loss - 0.6790537238, correct/total - 0/1024 (0.00%), bits/total - 26211/49152 (53

TRAIN epoch 18 iter 30: loss - 0.6765581369, correct/total - 0/1024 (0.00%), bits/total - 26547/49152 (54.01%). 
TRAIN epoch 18 iter 40: loss - 0.6769822836, correct/total - 0/1024 (0.00%), bits/total - 26466/49152 (53.85%). 
TRAIN epoch 18 iter 50: loss - 0.6764087677, correct/total - 0/1024 (0.00%), bits/total - 26544/49152 (54.00%). 
TRAIN epoch 18 completed! Avg train loss - 0.676808
	TRAIN data correct/total - 0/1021 (0.00%), bits correct/total - 26429/49027 (53.91%)
	VAL   data correct/total - 0/883 (0.00%), bits correct/total - 22768/42400 (53.70%)
TRAIN epoch 19 iter 0: loss - 0.6773409247, correct/total - 0/1024 (0.00%), bits/total - 26242/49152 (53.39%). 
TRAIN epoch 19 iter 10: loss - 0.6767746210, correct/total - 0/1024 (0.00%), bits/total - 26381/49152 (53.67%). 
TRAIN epoch 19 iter 20: loss - 0.6770246029, correct/total - 0/1024 (0.00%), bits/total - 26315/49152 (53.54%). 
TRAIN epoch 19 iter 30: loss - 0.6766726971, correct/total - 0/1024 (0.00%), bits/total - 26562/4915

TRAIN epoch 27 iter 40: loss - 0.6751660705, correct/total - 0/1024 (0.00%), bits/total - 26791/49152 (54.51%). 
TRAIN epoch 27 iter 50: loss - 0.6756856441, correct/total - 0/1024 (0.00%), bits/total - 26807/49152 (54.54%). 
TRAIN epoch 27 completed! Avg train loss - 0.675630
	TRAIN data correct/total - 0/1021 (0.00%), bits correct/total - 26671/49027 (54.40%)
	VAL   data correct/total - 0/883 (0.00%), bits correct/total - 22931/42400 (54.08%)
TRAIN epoch 28 iter 0: loss - 0.6753075123, correct/total - 0/1024 (0.00%), bits/total - 26899/49152 (54.73%). 
TRAIN epoch 28 iter 10: loss - 0.6756019592, correct/total - 0/1024 (0.00%), bits/total - 26711/49152 (54.34%). 
TRAIN epoch 28 iter 20: loss - 0.6756559610, correct/total - 0/1024 (0.00%), bits/total - 26738/49152 (54.40%). 
TRAIN epoch 28 iter 30: loss - 0.6752588749, correct/total - 0/1024 (0.00%), bits/total - 26731/49152 (54.38%). 
TRAIN epoch 28 iter 40: loss - 0.6751202941, correct/total - 0/1024 (0.00%), bits/total - 27020/4915

TRAIN epoch 36 iter 50: loss - 0.6740667224, correct/total - 0/1024 (0.00%), bits/total - 27131/49152 (55.20%). 
TRAIN epoch 36 completed! Avg train loss - 0.674679
	TRAIN data correct/total - 0/1021 (0.00%), bits correct/total - 26869/49027 (54.80%)
	VAL   data correct/total - 0/883 (0.00%), bits correct/total - 23100/42400 (54.48%)
TRAIN epoch 37 iter 0: loss - 0.6745320559, correct/total - 0/1024 (0.00%), bits/total - 27081/49152 (55.10%). 
TRAIN epoch 37 iter 10: loss - 0.6744215488, correct/total - 0/1024 (0.00%), bits/total - 27057/49152 (55.05%). 
TRAIN epoch 37 iter 20: loss - 0.6746363044, correct/total - 0/1024 (0.00%), bits/total - 26991/49152 (54.91%). 
TRAIN epoch 37 iter 30: loss - 0.6740920544, correct/total - 0/1024 (0.00%), bits/total - 27184/49152 (55.31%). 
TRAIN epoch 37 iter 40: loss - 0.6744291782, correct/total - 0/1024 (0.00%), bits/total - 26998/49152 (54.93%). 
TRAIN epoch 37 iter 50: loss - 0.6744092703, correct/total - 0/1024 (0.00%), bits/total - 27123/4915

...

TRAIN epoch 117 iter 40: loss - 0.6679121256, correct/total - 0/1024 (0.00%), bits/total - 28100/49152 (57.17%). 
TRAIN epoch 117 iter 50: loss - 0.6694669724, correct/total - 0/1024 (0.00%), bits/total - 27737/49152 (56.43%). 
TRAIN epoch 117 completed! Avg train loss - 0.667729
	TRAIN data correct/total - 0/1021 (0.00%), bits correct/total - 27955/49027 (57.02%)
	VAL   data correct/total - 0/883 (0.00%), bits correct/total - 23743/42400 (56.00%)
TRAIN epoch 118 iter 0: loss - 0.6677805781, correct/total - 0/1024 (0.00%), bits/total - 27965/49152 (56.89%). 
TRAIN epoch 118 iter 10: loss - 0.6680396795, correct/total - 0/1024 (0.00%), bits/total - 28048/49152 (57.06%). 
TRAIN epoch 118 iter 20: loss - 0.6670903563, correct/total - 0/1024 (0.00%), bits/total - 28181/49152 (57.33%). 
TRAIN epoch 118 iter 30: loss - 0.6682349443, correct/total - 0/1024 (0.00%), bits/total - 27986/49152 (56.94%). 
TRAIN epoch 118 iter 40: loss - 0.6663433313, correct/total - 0/1024 (0.00%), bits/total - 28

Process Process-3915:
Traceback (most recent call last):
Process Process-3918:
Process Process-3914:
Process Process-3916:
Process Process-3911:
Process Process-3913:
Process Process-3917:
Process Process-3909:
Process Process-3919:
Process Process-3912:
Process Process-3920:
Process Process-3910:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process Process-3907:
Traceback (

KeyboardInterrupt: 

  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in

  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/ubuntu/miniconda2/envs/cs670project/lib/python3.6/multiprocessing/